In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../orig-power-soton-page-flow.csv')

In [24]:
# Create a new transformed dataset with rows of source and target pages
df_transformed = pd.DataFrame(
    columns = ['user_number', 'session_number', 'session_start_time', 'source', 'source_duration', 'target']
)
column_names = df.columns.values.tolist()
new_row_index = 0
for index, row in df.iterrows():
    for column_name in column_names:
        if 'Unnamed' in column_name and not int(column_name[9:]) % 2 == 0 and pd.notnull(df.loc[index, column_name]):
            column_number = int(column_name[9:])
            if column_number <= 167:
                df_transformed.set_value(new_row_index, 'source', row[column_name])
                df_transformed.set_value(new_row_index, 'source_duration', row[column_name[:9] + str(column_number + 1)])
            if column_number <= 166:
                df_transformed.set_value(new_row_index, 'target', row[column_name[:9] + str(column_number + 2)])
            
            df_transformed.set_value(new_row_index, 'user_number', row['user number'])
            df_transformed.set_value(new_row_index, 'session_number', row['session number'])
            df_transformed.set_value(new_row_index, 'session_start_time', row['session start time'])
            
            new_row_index += 1

In [26]:
# dump transformed dataset to csv file
df_transformed.to_csv('./data/power-page-flow-transformed-data.csv')